<a href="https://colab.research.google.com/github/srilamaiti/fall_2023_210_capstone_deepika_srila/blob/main/final_deliverable/model/train_val_test_file_generation%20/generate_train_val_test_files_8classes_webscraped_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import random
import tqdm
import sys
import pathlib
import itertools
import collections
from pathlib import Path
from sklearn.utils import shuffle
import shutil
import glob
from glob import glob
import urllib.request
import subprocess
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
os.environ['GDRIVE_CONFIG_DIR'] = "/content/drive/MyDrive"
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

Mounted at /content/drive


In [2]:
LABEL_LIST = ['hello', 'nice', 'meet', 'you', 'thankyou', 'teacher', 'class', 'love']

label_info_dict = {}
for label in LABEL_LIST:
    label_info_dict[label] = {'gdrive_dir_path' : os.path.join(os.environ['GDRIVE_CONFIG_DIR'], 'webscraped_videos', f'{label}_avi_videos')}

SPLITS = (.6, .2, .2)
TOTAL_SAMPLES_PER_LABEL = 60
TRAIN_RECS_PER_LABEL = int(TOTAL_SAMPLES_PER_LABEL * SPLITS[0])
VAL_RECS_PER_LABEL = int(TOTAL_SAMPLES_PER_LABEL * SPLITS[1])
TEST_RECS_PER_LABEL = int(TOTAL_SAMPLES_PER_LABEL * SPLITS[2])
print(f"TRAIN_RECS_PER_LABEL : {TRAIN_RECS_PER_LABEL}")
print(f"VAL_RECS_PER_LABEL : {VAL_RECS_PER_LABEL}")
print(f"TEST_RECS_PER_LABEL : {TEST_RECS_PER_LABEL}")

TRAIN_FILE_LIST = []
VAL_FILE_LIST = []
TEST_FILE_LIST = []

def create_dir(dir_path):
    os.makedirs(dir_path, exist_ok=True)

def copy_file(file_name, dir_path):
    !cp {file} {dir_path}

def copy_set_of_files(file_list, dir_path):
    #print(len(file_list), dir_path)
    for file in file_list:
        #print(f"Copying {file} to {dir_path}")
        #copy_file(file_name = file, dir_path = dir_path)
        !cp {file} {dir_path}
    print(f"File count in {dir_path} : {len(os.listdir(dir_path))}")
    #print(os.listdir(dir_path))

def remove_files_from_dir(dir_path):
    for file in os.listdir(dir_path):
        qualified_file_name = os.path.join(dir_path, file)
        !rm -rf {qualified_file_name}
    print(f"File count in {dir_path} : {len(os.listdir(dir_path))}")

random.seed(1234)
os.environ['PYTHONHASHSEED'] = '1234'

# Saving files in gdrive
#current_ts = datetime.today().isoformat().replace("-","").replace("T","").replace(":","").replace(".","")
TRAIN_FILE_NAME = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '8classes_train_val_test_files_webscraped_videos', 'train_files.txt')
VAL_FILE_NAME = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '8classes_train_val_test_files_webscraped_videos', 'val_files.txt')
TEST_FILE_NAME = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '8classes_train_val_test_files_webscraped_videos', 'test_files.txt')

train_file_pointer = open(TRAIN_FILE_NAME, 'w')
val_file_pointer = open(VAL_FILE_NAME, 'w')
test_file_pointer = open(TEST_FILE_NAME, 'w')

for label in LABEL_LIST:
    gdrive_path = label_info_dict[label]['gdrive_dir_path']

    # test
    temp_test_files_1 = subprocess.getoutput(f"shuf -zn{TEST_RECS_PER_LABEL} -e {gdrive_path}/*.avi ").split("\n")
    test_files = [e.split(" -> ")[0].replace("'","") for e in temp_test_files_1]
    test_files = test_files[0].split("\x00")[:-1]

    # train
    temp_train_files_1 = list(set([os.path.join(gdrive_path, file) for file in os.listdir(gdrive_path)]) - set(test_files))
    temp_train_files_1 = sorted(temp_train_files_1, key=lambda x: random.random())
    train_files = random.sample(temp_train_files_1, TRAIN_RECS_PER_LABEL)

    # val
    temp_val_files_1 = list(set([os.path.join(gdrive_path, file) for file in os.listdir(gdrive_path)]) - set(test_files) - set(train_files))
    temp_val_files_1 = sorted(temp_val_files_1, key=lambda x: random.random())
    val_files = random.sample(temp_val_files_1, VAL_RECS_PER_LABEL)

    [train_file_pointer.writelines(e + "\n") for e in train_files]
    [val_file_pointer.writelines(e + "\n") for e in val_files]
    [test_file_pointer.writelines(e + "\n") for e in test_files]

# File closing
train_file_pointer.close()
val_file_pointer.close()
test_file_pointer.close()

TRAIN_RECS_PER_LABEL : 36
VAL_RECS_PER_LABEL : 12
TEST_RECS_PER_LABEL : 12


In [3]:
!cat {TRAIN_FILE_NAME}|wc -l

288


In [4]:
!cat {TEST_FILE_NAME}|wc -l

96


In [5]:
!cat {VAL_FILE_NAME}|wc -l

96


In [6]:
TRAIN_FILE_PATH = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '8classes_train_val_test_files_webscraped_videos', 'train')
VAL_FILE_PATH = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '8classes_train_val_test_files_webscraped_videos', 'val')
TEST_FILE_PATH = os.path.join(os.environ['GDRIVE_CONFIG_DIR'], '8classes_train_val_test_files_webscraped_videos', 'test')

for label in LABEL_LIST:
    train_dir_name = os.path.join(TRAIN_FILE_PATH, label)
    create_dir(train_dir_name)

    val_dir_name = os.path.join(VAL_FILE_PATH, label)
    create_dir(val_dir_name)

    test_dir_name = os.path.join(TEST_FILE_PATH, label)
    create_dir(test_dir_name)

In [7]:
def copy_file_to_folder(file_name, dir_path):
    fp = open(file_name, "r")
    for file in fp:
        file = file.strip()
        label = file.split("/")[-2].split("_")[0]
        base_file_name = os.path.basename(file)
        dest_path = os.path.join(dir_path, label)
        shutil.copy(file, dest_path)
        #!ls -ltrSh {dir_path}/{label}/*.avi | wc -l

In [8]:
for label in LABEL_LIST:
    !rm -rf {TRAIN_FILE_PATH}/{label}/*.avi
    !rm -rf {VAL_FILE_PATH}/{label}/*.avi
    !rm -rf {TEST_FILE_PATH}/{label}/*.avi

copy_file_to_folder(file_name = TRAIN_FILE_NAME, dir_path = TRAIN_FILE_PATH)
copy_file_to_folder(file_name = VAL_FILE_NAME, dir_path = VAL_FILE_PATH)
copy_file_to_folder(file_name = TEST_FILE_NAME, dir_path = TEST_FILE_PATH)

for label in LABEL_LIST:
    !ls -ltrSh {TRAIN_FILE_PATH}/{label}/*.avi | wc -l
    !ls -ltrSh {VAL_FILE_PATH}/{label}/*.avi | wc -l
    !ls -ltrSh {TEST_FILE_PATH}/{label}/*.avi | wc -l

36
12
12
36
12
12
36
12
12
36
12
12
36
12
12
36
12
12
36
12
12
36
12
12
